In [1]:
from database.market import Market
import pandas as pd
from extractor.tiingo_extractor import TiingoExtractor
from tqdm import tqdm

In [2]:
start = "2020-07-01"
end = "2020-09-30"
market = Market()

In [3]:
market.connect()
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",attrs={"id":"constituents"},header=0)[0]
market.store_data("sp500",df)
market.close()

In [8]:
market.connect()
tickers = market.retrieve_data("sp500")["Symbol"]
market.close()

In [11]:
market.connect()
fails = []
for ticker in tqdm(tickers):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        status = "tiingo"
        tex = TiingoExtractor(ticker,start,end)
        data = tex.extract()
        data["ticker"] = ticker
        market.store_data("prices",data)
    except Exception as e:
        msg = {"status":status,"ticker":ticker,"exception":str(e)}
        print(msg)
        fails.append(msg)
market.close()

 99%|████████████████████████████████████████████████████████████████████████████████████████████████▍| 502/505 [05:22<00:01,  2.87it/s]

If using all scalar values, you must pass an index
{'status': 'tiingo', 'ticker': 'YUM', 'exception': "'NoneType' object does not support item assignment"}
If using all scalar values, you must pass an index
{'status': 'tiingo', 'ticker': 'ZBRA', 'exception': "'NoneType' object does not support item assignment"}


100%|████████████████████████████████████████████████████████████████████████████████████████████████▊| 504/505 [05:22<00:00,  4.19it/s]

If using all scalar values, you must pass an index
{'status': 'tiingo', 'ticker': 'ZBH', 'exception': "'NoneType' object does not support item assignment"}
If using all scalar values, you must pass an index
{'status': 'tiingo', 'ticker': 'ZION', 'exception': "'NoneType' object does not support item assignment"}


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [05:22<00:00,  1.56it/s]

If using all scalar values, you must pass an index
{'status': 'tiingo', 'ticker': 'ZTS', 'exception': "'NoneType' object does not support item assignment"}


In [12]:
fails

[{'status': 'tiingo',
  'ticker': 'YUM',
  'exception': "'NoneType' object does not support item assignment"},
 {'status': 'tiingo',
  'ticker': 'ZBRA',
  'exception': "'NoneType' object does not support item assignment"},
 {'status': 'tiingo',
  'ticker': 'ZBH',
  'exception': "'NoneType' object does not support item assignment"},
 {'status': 'tiingo',
  'ticker': 'ZION',
  'exception': "'NoneType' object does not support item assignment"},
 {'status': 'tiingo',
  'ticker': 'ZTS',
  'exception': "'NoneType' object does not support item assignment"}]

In [19]:

for ticker in tqdm(fails):
    try:
        print(ticker)
        if "." in ticker:
            ticker = ticker.replace(".","-")
        tex = TiingoExtractor(ticker,start,end)
        data = tex.extract()
        data["ticker"] = ticker
        market.store_data("prices",data)
    except Exception as e:
        msg = {"status":status,"ticker":ticker,"exception":str(e)}
        fails.append(msg)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]

YUM
ZBRA
ZBH
ZION
ZTS


In [3]:
fails = ["YUM","ZBRA","ZBH","ZION","ZTS"]
ticker = "YUM"
tex = TiingoExtractor(ticker,start,end)
data = tex.extract()
# data["ticker"] = ticker
# market.store_data("prices",data)

In [7]:
data.content

b'You have run over your 500 symbol look up for this month. Please upgrade at https://api.tiingo.com/pricing to have your limits increased.'